In [20]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import networkx as nx
import os
import csv
from os import listdir
from os.path import isfile, join
from numpy import mean
from numpy import std
from scipy.stats import norm
from mpl_toolkits.mplot3d import Axes3D
from mpl_toolkits.mplot3d.art3d import Line3DCollection

In [28]:
# ==========================================================    
# Declaração de constantes
# ==========================================================  
CSV_DIR = "../csv/"
OUT_FILE = "metricas_redes_complexas.csv"
DIR = '../rt/'
DIC_SPEED = {
    "Static": 0,
    "IPv4SlowMobility": 1,
    "IPv4ModerateFastMobility": 2,
    "IPv4FastMobility": 3,
}

# ==========================================================    
# Cria um dataframe Pandas para guardar todos os cenários
# ==========================================================    
if os.path.isfile(join(CSV_DIR, OUT_FILE)):
    df_cenarios = pd.read_csv(join(CSV_DIR, OUT_FILE), low_memory = False)
else:
    df_cenarios = pd.DataFrame(columns = ["scenario", "speed", "hosts", "timeout", "round_idx", "snapshot", "avg_degree", "density", "avg_clustering_coef", "avg_shortest_path", "diameter", "arquivo"])

# ==========================================================    
# Varre todos os subdiretórios e processa os arquivos .rt
# ==========================================================    
for d in os.listdir(DIR):
    if (d != "IPv4SlowMobility_30_1"):
        continue    
    
    for f in os.listdir(join(DIR, d)):
        f = join(DIR, d, f)
        
        # ==========================================================
        # Verifica se o arquivo já foi processado
        # ==========================================================
        if (not df_cenarios.query('arquivo=="' + f + '"').empty):
            continue

        # ==========================================================
        # Lê as informações do cenário e rodada
        # ==========================================================
        scenario = d
        info = scenario.split("_")
        speed = DIC_SPEED[info[0]]
        info_hosts = info[1]
        timeout = info[2]
        round_idx = f.split("-")[1].replace(".rt", "")

        # ==========================================================
        # Limpa caracteres impróprios dos arquivos .rt
        # ==========================================================

        # Read in the file
        with open(f, 'r') as file :
          filedata = file.read()

        # Replace the target string
        filedata = filedata.replace('  ', ' ')
        filedata = filedata.replace('/', ' ')
        filedata = filedata.replace('s', '')
        filedata = filedata.replace('#', '')

        # Write the file out again
        with open(f, 'w') as file:
          file.write(filedata)

        # ==========================================================
        # Lê o arquivo como um dataframe Pandas
        # ==========================================================
        columns = ['acao', 'evento', 'timestamp', 'tabela', 'origem', 'destino', 'mascara', 'gateway', 'interface']
        aodv = pd.read_csv(f, names = columns, sep = ' ')
        hosts = aodv[aodv.destino == '127.0.0.0']['origem']
        eventos = aodv[aodv.timestamp != 0][['acao', 'timestamp', 'origem', 'destino', 'gateway']]

        # ==========================================================
        # Processa a tabela de rotamento com variação no tempo
        # ==========================================================
        start = 0 # início da simulação
        stop = 200 # condição de parada em segundos
        snapshot = 1000 # intervalo de amostragem do snapshot (em milisegundos)

        t = start
        while t <= stop:
            rotas = []
            for idx, e in eventos[eventos.timestamp <= t].iterrows():
                cria = False
                rota = next((r for r in rotas if r['origem'] == e['origem'] and r['destino'] == e['destino']), None)
                if rota == None:
                    cria = True
                    rota = {'origem': e['origem'], 'destino': e['destino'], 'gateway': e['gateway']}
                if '+' in e['acao'] and cria: # criação de nova rota
                    rotas.append(rota)
                elif '*' in e['acao']: # alteração de rota existente
                    rota.update({'gateway': e['gateway']})
                else: # removação de rota existente
                    rotas.remove(rota)

            # construção do grafo
            G = nx.DiGraph()
            c = 1
            for h in hosts:
                G.add_node(h, **{'c':c})
                c += 1
            for r in rotas:
                G.add_edge(r['origem'], r['gateway'])

            # ==========================================================
            # Cálculo das métricas e geração do dataframe Pandas
            # ==========================================================
            try:
                sp = nx.average_shortest_path_length(G)
            except:
                sp = None
            try:
                diam = nx.diameter(G)
            except:
                diam = None
            dg = []
            for n, degree in nx.degree(G):
                dg.append(degree)

            data = pd.DataFrame(
                {"scenario": scenario,
                 "speed": pd.to_numeric(speed),
                 "hosts": pd.to_numeric(info_hosts),
                 "timeout": pd.to_numeric(timeout),
                 "round_idx": pd.to_numeric(round_idx),
                 "snapshot": t, 
                 "avg_degree": mean(dg), 
                 "density": nx.density(G), 
                 "avg_clustering_coef": nx.average_clustering(G),
                 "avg_shortest_path": sp,
                 "diameter": diam,
                 "arquivo": f
                },
                index=[0]
            )
            df_cenarios = df_cenarios.append(data, ignore_index = True)
            t += snapshot / 1000

        # Salva dataframe em CSV
        df_cenarios.to_csv(join(CSV_DIR, OUT_FILE))

KeyboardInterrupt: 